In [1]:
import json
from dataclasses import dataclass
from typing import Optional

@dataclass
class PatientData:
    question2: int  # Age
    question3: str # Gender "Item 1" for زن (female), "Item 2" for مرد (male)
    question6: Optional[float] = None  # Height (cm)
    question7: Optional[float] = None  # Weight (kg)
    question4: Optional[str] = None  # Current smoker? "Item 2" for بله (yes)
    question29: Optional[list[str]] = None  # Osteoporosis risk factors checkboxes
    question26: Optional[str] = None # Menopause status for women < 65


def calculate_bmi(height_cm: Optional[float], weight_kg: Optional[float]) -> Optional[float]:
    """
    Calculates BMI given height in centimeters and weight in kilograms.
    Returns BMI or None if height or weight is missing or invalid.
    """
    if height_cm is None or weight_kg is None or height_cm <= 0 or weight_kg <= 0:
        return None
    height_m = height_cm / 100.0  # Convert cm to meters
    bmi = weight_kg / (height_m ** 2)
    return bmi


def has_osteoporosis_risk_factors(patient_data: PatientData) -> bool:
    """
    Checks if the patient has any osteoporosis risk factors based on the questionnaire data,
    including BMI for underweight and considering menopause for women under 65.
    """
    risk_factors = False

    # Underweight (BMI < 18.5 kg/m²)
    bmi = calculate_bmi(patient_data.question6, patient_data.question7)
    if bmi is not None and bmi < 18.5:
        risk_factors = True

    # Current smoking
    if patient_data.question4 == "Item 2": # "Item 2" for بله (yes) to "آیا در حال حاضر سیگار مصرف میکنید ؟"
        risk_factors = True

    # Checkbox question 29 for other risk factors
    if patient_data.question29:
        osteoporosis_specific_risks = patient_data.question29
        if any(item in osteoporosis_specific_risks for item in [
            "Item 1", # "شکستگی قبلی با وجود ضربه مختصر" (Previous fracture with minor trauma)
            "Item 2", # "شکستگی لگن در والدین" (Parental history of hip fracture)
            "Item 3", # "مصرف الکل" (Excess alcohol intake)
            "Item 4", # "بیماری روماتیسم مفاصل" (Rheumatoid arthritis)
            "Item 5", # "یائسگی زودرس" (Early menopause)
            "Item 6", # "سو جذب مواد غذایی" (Gastrointestinal malabsorption)
            "Item 7", # "بیماری مزمن کبدی ( مثل هپاتیت)" (Chronic liver disease)
            "Item 8", # "بیماری التهابی روده مثل کرون یا IBS" (Inflammatory bowel disease)
            "Item 9"  # "دریافت مکرر کورتون" (Long-term glucocorticoid therapy)
        ]):
            risk_factors = True

    return risk_factors


def check_osteoporosis_screening_indication(patient_data: PatientData) -> str:
    """
    Checks osteoporosis screening indication based on guidelines and patient data,
    considering menopause for women under 65 with risk factors.
    Returns a string indicating the screening strategy.
    """
    age = patient_data.question2
    gender = patient_data.question3
    has_risks = has_osteoporosis_risk_factors(patient_data)

    if gender == "Item 1": # زن (Female)
        if age >= 65:
            return "Patient is indicated for osteoporosis screening with DXA, routine frequency."
        elif age < 65 and has_risks and patient_data.question26 == "Item 2": # Woman < 65, with risks, AND postmenopausal
            return "Patient is indicated for osteoporosis screening with DXA, routine frequency."
        else:
            return "Patient is not indicated for osteoporosis screening based on current guidelines."
    elif gender == "Item 2": # مرد (Male)
        if age >= 70: # Based on BHOF, ISCD, Endocrine Society, Canadian Osteoporosis Society
            return "Patient is indicated for osteoporosis screening with DXA, routine frequency."
        elif 50 <= age <= 70 and has_risks: # Men 50-70 with risk factors (BHOF, ISCD, Endocrine Society)
            return "Patient is indicated for osteoporosis screening with DXA, routine frequency."
        else:
            return "Patient is not indicated for osteoporosis screening based on current guidelines."
    else:
        return "Could not determine screening indication due to missing gender information."



if __name__ == "__main__":
    # Example Patient Data Scenarios:

    # Woman under 65 with risk factors but NOT menopausal
    patient_data_1 = PatientData(
        question2=55,
        question3="Item 1",
        question6=160,
        question7=50,
        question4="Item 1",
        question29=["Item 2", "Item 9"],
        question26="Item 1" # "Item 1" for خیر (No) to "آیا شما دچار یائسگی شده اید ؟"
    )

    # Woman under 65 with risk factors AND menopausal
    patient_data_2 = PatientData(
        question2=60,
        question3="Item 1",
        question6=160,
        question7=50,
        question4="Item 1",
        question29=["Item 2", "Item 9"],
        question26="Item 2" # "Item 2" for بله (Yes) to "آیا شما دچار یائسگی شده اید ؟"
    )

    # Underweight person
    patient_data_3 = PatientData(
        question2=50,
        question3="Item 1",
        question6=170,
        question7=45, # weight for underweight BMI
        question4="Item 1",
        question29=[],
        question26="Item 2"
    )

    # Woman >= 65, no risk factors
    patient_data_4 = PatientData(
        question2=68,
        question3="Item 1",
        question6=165,
        question7=65,
        question4="Item 1",
        question29=[],
        question26="Item 2" # Doesn't matter for age >= 65
    )

    # Man >= 70, no risk factors
    patient_data_5 = PatientData(
        question2=72,
        question3="Item 2",
        question6=175,
        question7=80,
        question4="Item 1",
        question29=[]
    )

    # Man 50-70 with risk factors (smoking)
    patient_data_6 = PatientData(
        question2=60,
        question3="Item 2",
        question6=178,
        question7=90,
        question4="Item 2", # Current smoker
        question29=[]
    )

     # Man 50-70 with no risk factors
    patient_data_7 = PatientData(
        question2=60,
        question3="Item 2",
        question6=178,
        question7=90,
        question4="Item 1", # Not smoker
        question29=[]
    )

    patient_scenarios = [
        patient_data_1,
        patient_data_2,
        patient_data_3,
        patient_data_4,
        patient_data_5,
        patient_data_6,
        patient_data_7,
    ]

    for i, patient_data in enumerate(patient_scenarios):
        screening_recommendation = check_osteoporosis_screening_indication(patient_data)
        print(f"Scenario {i+1}:")
        print(f"Patient Data: {patient_data}")
        print(f"Screening Recommendation: {screening_recommendation}")
        print("-" * 30)

Scenario 1:
Patient Data: PatientData(question2=55, question3='Item 1', question6=160, question7=50, question4='Item 1', question29=['Item 2', 'Item 9'], question26='Item 1')
Screening Recommendation: Patient is not indicated for osteoporosis screening based on current guidelines.
------------------------------
Scenario 2:
Patient Data: PatientData(question2=60, question3='Item 1', question6=160, question7=50, question4='Item 1', question29=['Item 2', 'Item 9'], question26='Item 2')
Screening Recommendation: Patient is indicated for osteoporosis screening with DXA, routine frequency.
------------------------------
Scenario 3:
Patient Data: PatientData(question2=50, question3='Item 1', question6=170, question7=45, question4='Item 1', question29=[], question26='Item 2')
Screening Recommendation: Patient is indicated for osteoporosis screening with DXA, routine frequency.
------------------------------
Scenario 4:
Patient Data: PatientData(question2=68, question3='Item 1', question6=165, 